In [ ]:
import os
import datetime
import asyncio

from dotenv import load_dotenv
from notion_client import AsyncClient

from startpage.components.weather import get_weather
from startpage.components.currency import get_currency_rates
from startpage.utils.blocks import append_block_to_page, create_header_1_block, create_divider_block
from startpage.components.fact import get_random_fact
from startpage.components.calendar import get_icloud_calendar_events
from startpage.components.rss import fetch_feed, Feed

load_dotenv()

notion = AsyncClient(auth=os.environ["NOTION_TOKEN"])
page_id = os.environ["PAGE_ID"]
city = os.environ["CITY"]
username = os.environ['ICLOUD_USERNAME']  # your Apple ID
password = os.environ['ICLOUD_APP_PASSWORD']  # App-specific password
timezone = os.environ.get('TIMEZONE', 'UTC')

In [ ]:
now = datetime.datetime.now()
formatted_date = now.strftime("%A %d of %B")

feeds = [
    Feed(name="TechCrunch", url="https://techcrunch.com/feed/", priority=2),
    Feed(name="BigThinking", url="https://bigthinking.io/feed", priority=1),
    Feed(name="Product Hunt", url="https://www.producthunt.com/feed", priority=2),
    Feed(name="Hacker News Launches", url="https://news.ycombinator.com/launches", priority=1),
    Feed(name="Andrew Chen", url="https://andrewchen.substack.com/feed", priority=1),
    Feed(name="Benedict Evans", url="http://ben-evans.com/benedictevans?format=rss", priority=1),
    Feed(name="Irrational Exuberance", url="https://irrationalexuberance.libsyn.com/rss", priority=1),
    Feed(name="Pragmatic Engineer", url="https://pragmaticengineer.com/feed/", priority=1)
]
banned_tags={"sponsored", "advertisement", "government & policy", "smartglasses"}

weather_task = get_weather(city)
currency_task = get_currency_rates("Currencies (₽)", "rub", ["usd", "eur"])
crypto_task = get_currency_rates("Cryptocurrencies ($)", "usd", ["btc", "eth"])
calendar_task = get_icloud_calendar_events("Today's Events", username=username, password=password, timezone=timezone)
rss_task = fetch_feed("Tech News", feeds, banned_tags=banned_tags, max_articles=5)
fact_task = get_random_fact()

results = await asyncio.gather(
    weather_task,
    currency_task,
    crypto_task,
    calendar_task,
    rss_task,
    fact_task
)

weather_block, currency_blocks, crypto_blocks, calendar_events, rss_block, fact = results

divider = create_divider_block()
children = [*weather_block]
children.extend(currency_blocks)
children.extend(crypto_blocks)
children.extend(calendar_events)
children.extend(rss_block)
new_day = create_header_1_block(formatted_date, children)

block_id = os.environ["BLOCK_ID"]

await append_block_to_page(notion, page_id, new_day, after=block_id)
await notion.blocks.update(block_id, callout={"rich_text": [{"text": {"content": fact}}]})

Fetching events from calendar: Фестивали и праздники
Fetching events from calendar: Booking
Fetching events from calendar: Напоминания ⚠️
Fetching events from calendar: Семья
Fetching events from calendar: Семья
Fetching events from calendar: Домашний
Fetching events from calendar: Рабочий
Selected article: Redlight Greenlight for Claude Code from feed: Product Hunt


{'object': 'block',
 'id': '2d319fe9-9716-80e8-bca6-db82e1d6acdc',
 'parent': {'type': 'page_id',
  'page_id': '2d219fe9-9716-80e3-957a-c143d70f618b'},
 'created_time': '2025-12-24T12:23:00.000Z',
 'last_edited_time': '2026-01-14T09:21:00.000Z',
 'created_by': {'object': 'user',
  'id': 'f712041f-9fae-455e-8027-fa6c58ad5b52'},
 'last_edited_by': {'object': 'user',
  'id': 'f18583a3-7435-43a9-b9fd-1e7b11f1d6d2'},
 'has_children': False,
 'archived': False,
 'in_trash': False,
 'type': 'callout',
 'callout': {'rich_text': [{'type': 'text',
    'text': {'content': '100% of lottery winners do gain weight',
     'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': '100% of lottery winners do gain weight',
    'href': None}],
  'icon': {'type': 'emoji', 'emoji': '💡'},
  'color': 'gray_background'},
 'request_id': '6960842d-d4db-404f-85d0-fb7e43cec927'}